In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor


2022-11-01 14:30:37.246343: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-01 14:30:37.470834: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-01 14:30:37.470866: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-01 14:30:37.539523: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-01 14:30:38.786068: W tensorflow/stream_executor/platform/de

In [2]:
df = pd.read_csv('../datasets/ETHUSD_5m.csv' , names =['a' ,'unix','date','symbol','open','high','low','close','Volume_USD','Volume_crypto', 'k'] )
df = df.drop(['a' , 'k' , 'unix' ,'symbol' ] , axis = 1)
df

,date,open,high,low,close,Volume_USD,Volume_crypto
0,2018-06-01 00:06:00,579.08,579.56,577.52,579.48,6.880892e+00,3.985484e+03
1,2018-06-01 00:11:00,579.18,581.79,578.75,579.43,1.196595e+03,6.936516e+05
2,2018-06-01 00:16:00,579.32,580.72,578.95,579.30,1.217289e+03,7.056476e+05
3,2018-06-01 00:21:00,579.52,580.05,578.69,579.52,1.268806e+03,7.355171e+05
4,2018-06-01 00:26:00,580.60,581.08,579.83,581.08,1.326577e+03,7.690636e+05
...,...,...,...,...,...,...,...
308066,2021-06-01 23:36:00,2629.66,2634.34,2623.43,2626.11,5.471072e+07,3.461623e+10
308067,2021-06-01 23:41:00,2634.48,2637.12,2631.97,2633.88,5.471088e+07,3.461663e+10
308068,2021-06-01 23:46:00,2638.62,2650.00,2634.58,2641.46,5.471106e+07,3.461712e+10
308069,2021-06-01 23:51:00,2628.09,2639.03,2628.09,2628.09,5.471110e+07,3.461722e+10


In [3]:
cand = 30

In [4]:
df = df.dropna()

In [5]:
df = df.loc[:, df.columns != 'date']
df = df.values

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = scaler.fit_transform(df)

In [7]:
X = []
y = []
counter = 0 
for i in range(len(df)):
    temp = []
    counter = counter + 1
    for j in range(cand):
        temp.append(df[i+j])
    X.append(temp)
    y.append(df[i+cand][3])
        

IndexError: index 308071 is out of bounds for axis 0 with size 308071

In [8]:
X = np.array(X)
y = np.array(y)

In [9]:
X = X[:len(y)]

In [24]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

In [58]:
def baseline_model():
    model = Sequential()
    model.add(LSTM(32, input_shape=((cand, 6)), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(LSTM(16, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(LSTM(8))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mean_absolute_error'])
    return model

In [59]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=1, batch_size=20)

/tmp/ipykernel_6547/304737239.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasRegressor(build_fn=baseline_model, epochs=1, batch_size=20)


In [78]:
history=estimator.fit(train_X,train_y)

10320/10320 [==============================] - 750s 71ms/step - loss: 0.0826 - mean_absolute_error: 0.1697


In [79]:
y_pred = estimator.predict(test_X)

5083/5083 [==============================] - 91s 17ms/step


In [80]:
y_pred.shape[0]

101654

In [81]:
forinverse = np.zeros((y_pred.shape[0] , 6))
for i in range(len(forinverse)):
    forinverse[i][3] = y_pred[i]

In [82]:
forinverse1 = np.zeros((y_pred.shape[0] , 6))
for i in range(len(forinverse)):
    forinverse1[i][3] = test_y[i]

In [83]:
inversepred = scaler.inverse_transform(forinverse)
inversettrue = scaler.inverse_transform(forinverse1)

In [84]:
inversepred[: , 3]

array([ 198.1849965 ,  347.49488821,  155.15849126, ..., 2109.15074698,
        346.6843568 ,  179.47666851])

In [76]:
from sklearn.metrics import r2_score
r2_score(y_pred=inversepred[: , 3], y_true = inversettrue[: , 3])

0.9913365402347034

In [77]:
from sklearn.metrics import mean_absolute_error
colab(y_true = inversettrue[: , 3], y_pred = inversepred[: , 3])

43.67291314550709